In [1]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
weight_file = ''
cfg_file = ''
name_file = ''

min_confidence = 0.5

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import IPython
import time

In [ ]:
frame_count = 0
writer = None
input_name = 'traffic.mp4'
output_name = 'traffic.avi'
file_name = '00000.ppm'

In [ ]:
#load yolo
net = cv2.dnn.readNet(weight_file, cfg_file)

layer_names = net.getLayerNames()
output_layers = [layer_name[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
classes = []
with open(name_file, 'r') as f:
  classes = [line.strip() for line in f.readlines()]
print(classes)

In [ ]:
img = cv2.imread(file_name)
height, width, channels = img.shape

blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0,0,0), True, crop=Fasle)

net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []

for out in outs:
  for detection in out:
    scores = detection[5:]
    class_id = np.argmax(scores)
    confidence = scores[class_id]
    if confidence > min_confidence:
      #object detected
      center_x = int(detection[0]*width)
      center_y = int(detection[1]*height)
      w = int(detection[2]*width)
      h = int(detection[3]*height)

      #rectangle coordinates
      x = int(center_x - w / 2)
      y = int(center_y - h / 2)

      boxes.append([x, y, w, h])
      confidences.append(float(confidence))
      class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
font = cv2.FONT_HERSHEY_COMLEX

for i in range(len(boxes)):
  if i in indexes:
    x,y,w,h = boxes[i]
    label = str(classes[class_ids[i]])
    print(class_id[i], label)
    cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
    cv2.putText(img, label, (x, y-10), font, 0.5, (0,255,0),1)

cv2_imshow(img)

In [ ]:
def writeFrame(img):
  global writer
  height, width = img.shape[:2]
  if writer is None and output_name is not None:
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    writer = cv2.VideoWriter(output_name, fourcc, 24, (width, height), True)
  if writer is not None:
    writer.write(img)

In [ ]:
def detectAndDisplay(frame):
  global frame_count
  frame_count += 1
  start_time = time.time()
  IPython.display.clear_output(wait=True)
  height, width, channels = frame.shape

  blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0,0,0), True, crop=Fasle)

  net.setInput(blob)
  outs = net.forward(output_layers)

  class_ids = []
  confidences = []
  boxes = []

  for out in outs:
    for detection in out:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]
      if confidence > min_confidence:
        #object detected
        center_x = int(detection[0]*width)
        center_y = int(detection[1]*height)
        w = int(detection[2]*width)
        h = int(detection[3]*height)

        #rectangle coordinates
        x = int(center_x - w / 2)
        y = int(center_y - h / 2)

        boxes.append([x, y, w, h])
        confidences.append(float(confidence))
        class_ids.append(class_id)

  indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
  font = cv2.FONT_HERSHEY_COMLEX

  for i in range(len(boxes)):
    if i in indexes:
      x,y,w,h = boxes[i]
      label = str(classes[class_ids[i]])
      
      cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
      cv2.putText(img, label, (x, y+30), font, 0.5, (0,255,0),1)

  frame_time = time.time() - start_time
  print("Frame {} time {}".format(frame_count, frame_time))
  cv2_imshow(frame)
  writeFrame(frame)


In [ ]:
cap = cv2.VideoCapture(input_name)
if not cap.isOpened:
  print('--(!) Error opening video caputre')
  exit(0)
while True:
  ret, frame = cap.read()
  if frame is None:
    print('--(!) No captured frame -- break')
    break
  detectAndDisplay(frame)

In [ ]:
from google.colab import files
files.download(output_name)